In [2]:
import pandas as pd
import sqlite3 
import urllib
from bs4 import BeautifulSoup 
import sqlite3 
import time
from urllib.request import Request, urlopen
from IPython.display import clear_output

In [3]:
# Load package for progress bar
%run "progress_bar.py"
print ('Progress Bar Loaded')

Progress Bar Loaded


In [4]:
con = sqlite3.connect("pitchfork-data.db")
links_table = pd.read_sql_query("SELECT album, artist, score, link from album_table", con)

In [6]:
iterator = 0 #Set iterator to measure progress
con = sqlite3.connect("pitchfork-data.db") #create connection

for link in log_progress(links_table["link"][8363:], every=1):
    t0 = time.time()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    }
    req = Request(link, headers = headers)
    try: 
        webpage = urlopen(req).read()
        response_delay = time.time() - t0
        #time.sleep(response_delay)  # wait Nx longer than it took them to respond
        soup = BeautifulSoup(webpage, "lxml") #same as above
        #Pull in the components we are interested in
        score_soup = soup.findAll('div', {"class":"score-circle"}) #Score
        abstract_soup = soup.findAll('div', {"class":"review-detail__abstract"})
        review_soup = soup.findAll('div', {"class":"contents dropcap"})
        author_name_soup = soup.findAll('a', {"class":"authors-detail__display-name"})
        author_type_soup = soup.findAll('span', {"class":"authors-detail__title"})
        #Loop each subset of the soup to grab the relevant info
        for score_value in score_soup:
            score = score_value.text
        for abstract in abstract_soup:
            abstract_text = abstract.text
        for review in review_soup:
            review_text = review.text
        for author in author_name_soup:
            author_name = author.text
        for author in author_type_soup:
            author_type = author.text
        clear_output()
        print ("Author:", author_name,
               "\nScore:", score,
               "\nLink:", link, 
               "\nDelay:", round(response_delay, 2), 
               "\nIterator:", iterator)
        #Create the table
        table = pd.DataFrame({'author': author_name, 
                              'author_type'  : author_type,
                              'score' : score,
                              'abstract': abstract_text,
                              'review' : review_text,
                              'link' : link},
                              index=[0])
        table.to_sql("review_text",
                     con,if_exists = "append", 
                                         index = False)
        iterator= iterator + 1
    except KeyboardInterrupt:
        raise
    except:
        pass

con.close() 

Author: Jason Crock 
Score: 7.5 
Link: http://www.pitchfork.com/reviews/albums/13940-tapestry-of-webs/ 
Delay: 0.82 
Iterator: 7261


KeyboardInterrupt: 